In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import os
import zipfile
import urllib.request
import time

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Step 1: Download TinyImageNet dataset (subset of ImageNet)
url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
data_dir = './data/tiny-imagenet-200'

if not os.path.exists(data_dir):
    print("Downloading TinyImageNet dataset...")
    urllib.request.urlretrieve(url, './tiny-imagenet-200.zip')

    # Extracting the dataset
    with zipfile.ZipFile('./tiny-imagenet-200.zip', 'r') as zip_ref:
        zip_ref.extractall('./data')
    print("Dataset downloaded and extracted.")

# Step 2: Define data transformations
input_size = 224
batch_size = 32

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Step 3: Load datasets
data_dir = './data/tiny-imagenet-200'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

# TinyImageNet has no explicit train/val split; let's create one
full_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])

# Use only the first 80 classes for your use case
class_subset = 80
targets = torch.tensor([sample[1] for sample in full_dataset.samples])
indices = [i for i in range(len(targets)) if targets[i] < class_subset]

# Create subset dataset
subset_dataset = torch.utils.data.Subset(full_dataset, indices)

# Split the subset into training and validation (80% train, 20% val)
train_size = int(0.8 * len(subset_dataset))
val_size = len(subset_dataset) - train_size
train_dataset, val_dataset = random_split(subset_dataset, [train_size, val_size])

# Create DataLoaders for training and validation datasets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Step 4: Define ResNet-50 model
model = models.resnet50(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, class_subset)  # Modify for 80 classes
model = model.to(device)

# Step 5: Define loss function, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Step 6: Profiling clock cycles (using time module)
def train_model_with_profiling(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10):
    total_start_time = time.time()
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 20)

        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        epoch_start_time = time.time()

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        epoch_duration = time.time() - epoch_start_time
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        print(f'Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_acc:.4f}')
        print(f'Epoch {epoch + 1} duration: {epoch_duration:.4f} seconds')

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_epoch_loss = val_loss / len(val_loader.dataset)
        val_epoch_acc = val_correct / val_total
        print(f'Val Loss: {val_epoch_loss:.4f}, Val Accuracy: {val_epoch_acc:.4f}')

        scheduler.step()

    total_duration = time.time() - total_start_time
    print(f'Total training time: {total_duration:.4f} seconds')
    return total_duration

# Start training and measure time
total_training_time = train_model_with_profiling(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10)

# Assuming the CPU clock speed of Google Colab is ~2.3 GHz (check your instance using `!lscpu`)
cpu_clock_speed_ghz = 2.3
total_clock_cycles = total_training_time * cpu_clock_speed_ghz * 1e9
print(f"Estimated Total Clock Cycles: {total_clock_cycles:.2e} cycles")


Using device: cuda
Epoch 1/10
--------------------
